## Telegram interactor

In [1]:
from telethon import TelegramClient
import asyncio
import dotenv

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import os
# Your API credentials (get from my.telegram.org)
api_id = os.environ["TG_APP_ID"]
api_hash = os.environ["TG_APP_HASH"]

client = TelegramClient('session_name', api_id, api_hash)

In [35]:
await client.start()

In [4]:
import os
os.environ["TG_APP_ID"]

'23770916'

In [29]:
messages = await client.get_messages('@solana_trojanbot')

In [30]:
messages

[<telethon.tl.patched.Message object at 0x107e16ed0>, total=56]

In [ ]:
await messages[0].click(1, 0)

In [36]:
message = messages[0]

In [37]:
    # Check if message has buttons
    if message.buttons:
        for row_index, row in enumerate(message.buttons):
            for col_index, button in enumerate(row):
                print(f"Button [{row_index}][{col_index}]: {button.text}")
                
                # Check if button is clickable
                if hasattr(button, 'click'):
                    print(f"  - Clickable: Yes")
                    # You can click it like this:
                    # await button.click()
                else:
                    print(f"  - Clickable: No")

Button [0][0]: Buy
  - Clickable: Yes
Button [0][1]: Sell
  - Clickable: Yes
Button [1][0]: Positions
  - Clickable: Yes
Button [1][1]: Limit Orders
  - Clickable: Yes
Button [1][2]: DCA Orders
  - Clickable: Yes
Button [2][0]: Copy Trade
  - Clickable: Yes
Button [2][1]: Sniper 🆕
  - Clickable: Yes
Button [3][0]: Trenches
  - Clickable: Yes
Button [3][1]: 💰 Rewards
  - Clickable: Yes
Button [3][2]: ⭐️ Watchlist
  - Clickable: Yes
Button [4][0]: Withdraw
  - Clickable: Yes
Button [4][1]: Settings
  - Clickable: Yes
Button [5][0]: Help
  - Clickable: Yes
Button [5][1]: ↻ Refresh
  - Clickable: Yes


In [45]:
await client.disconnect()

In [46]:
messages = await client.get_messages('@dezhurka_hell')

ConnectionError: Cannot send requests while disconnected

In [39]:
messages

[<telethon.tl.patched.Message object at 0x118130190>, total=105]

In [40]:
message = messages[0]

In [44]:
message.

TypeError: 'MessageReplies' object is not subscriptable

In [10]:
import threading
import time
import traceback
import websocket
import json
import os

# Message handling callback
def on_message(ws, message):
    try:
        print(f"\nReceived message: {message}")
        # Convert to JSON
        result_json = json.loads(message)
        event_type = result_json.get("event_type")
        
        if event_type == "connected":
            print("Connection successful!")
            return
        if event_type == "ping":
            print("ping!")
            timestamp = result_json.get("timestamp")
            current_time_ms = time.time() * 1000
            current_time_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

            # Calculate and format time difference
            diff_time_ms = current_time_ms - timestamp
            diff_time_seconds = diff_time_ms / 1000
            diff_time_formatted = f"{int(diff_time_seconds // 60)}min{int(diff_time_seconds % 60)}sec"

            # Format original timestamp
            timestamp_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp/1000))

            # Print information
            print(f"Current time: {current_time_str}")
            print(f"Message timestamp: {timestamp_str}")
            print(f"Time difference: {diff_time_formatted} ({diff_time_ms:.0f} milliseconds)")
            return
        
        if event_type == "tweet":
            print("tweet!")
            # Extract fields
            rule_id = result_json.get("rule_id")
            rule_tag = result_json.get("rule_tag")
            event_type = result_json.get("event_type")
            tweets = result_json.get("tweets", [])
            timestamp = result_json.get("timestamp")
            
            # Print key information
            print(f"rule_id: {rule_id}")
            print(f"rule_tag: {rule_tag}")
            print(f"event_type: {event_type}")
            print(f"Number of tweets: {len(tweets)}")
            print(f"timestamp: {timestamp}")
            
            # Calculate time difference
            current_time_ms = time.time() * 1000
            current_time_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
            diff_time_ms = current_time_ms - timestamp
            diff_time_seconds = diff_time_ms / 1000
            diff_time_formatted = f"{int(diff_time_seconds // 60)}min{int(diff_time_seconds % 60)}sec"
            timestamp_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(timestamp/1000))

            print(f"Current time: {current_time_str}")
            print(f"Message timestamp: {timestamp_str}")
            print(f"Time difference: {diff_time_formatted} ({diff_time_ms:.0f} milliseconds)")
        
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}. traceback: {traceback.format_exc()}")
    except Exception as e:
        print(f"Error occurred while processing message: {e}. traceback: {traceback.format_exc()}")

# Error handling callback
def on_error(ws, error):
    print(f"\nError occurred: {error}, traceback: {traceback.format_exc()}")
    
    if isinstance(error, websocket.WebSocketTimeoutException):
        print("Connection timeout. Please check if server is running or network connection.")
    elif isinstance(error, websocket.WebSocketBadStatusException):
        print(f"Server returned error status code: {error}")
        print("Please check if API key and endpoint path are correct.")
    elif isinstance(error, ConnectionRefusedError):
        print("Connection refused. Please confirm server address and port are correct.")

# Connection close callback
def on_close(ws, close_status_code, close_msg):
    print(f"\nConnection closed: status_code={close_status_code}, message={close_msg}")
    
    if close_status_code == 1000:
        print("Normal connection closure")
    elif close_status_code == 1001:
        print("Server is shutting down or client navigating away")
    elif close_status_code == 1002:
        print("Protocol error")
    elif close_status_code == 1003:
        print("Received unacceptable data type")
    elif close_status_code == 1006:
        print("Abnormal connection closure, possibly network issues")
    elif close_status_code == 1008:
        print("Policy violation")
    elif close_status_code == 1011:
        print("Server internal error")
    elif close_status_code == 1013:
        print("Server overloaded")

# Connection established callback
def on_open(ws):
    print("\nConnection established!")

def connect_websocket(url, headers):
    while True:
        try:
            ws = websocket.WebSocketApp(
                url,
                header=headers,
                on_message=on_message,
                on_error=on_error,
                on_close=on_close,
                on_open=on_open
            )
            
            # No automatic reconnect, handle manually
            ws.run_forever(ping_interval=30, ping_timeout=20)
            
        except KeyboardInterrupt:
            break
        except Exception as e:
            logging.error(f"Connection error: {e}")
            time.sleep(5)  # Quick retry for network issues
            continue

# Main function
def main(x_api_key):
    url = "wss://ws.twitterapi.io/twitter/tweet/websocket"
    headers = {"x-api-key": x_api_key}
    
    connect_websocket(url, headers)

In [9]:
main(os.environ["TWITTERAPI_KEY"])


Connection established!

Received message: {"status": "connected", "event_type": "connected", "timestamp": 1752959418492, "user_id": "337268830288183296"}
Connection successful!

Received message: {"event_type": "ping", "timestamp": 1752959438704}
ping!
Current time: 2025-07-19 16:10:36
Message timestamp: 2025-07-19 16:10:38
Time difference: -1min58sec (-1969 milliseconds)

Received message: {"event_type": "ping", "timestamp": 1752959458920}
ping!
Current time: 2025-07-19 16:10:56
Message timestamp: 2025-07-19 16:10:58
Time difference: -1min58sec (-1963 milliseconds)

Received message: {"event_type": "ping", "timestamp": 1752959479136}
ping!
Current time: 2025-07-19 16:11:17
Message timestamp: 2025-07-19 16:11:19
Time difference: -1min58sec (-1969 milliseconds)

Received message: {"event_type": "ping", "timestamp": 1752959499353}
ping!
Current time: 2025-07-19 16:11:37
Message timestamp: 2025-07-19 16:11:39
Time difference: -1min58sec (-1937 milliseconds)

Received message: {"event_ty